In [ ]:
import os
import os.path as osp
import numpy as np

path = '../hot3dgaze'
#quat: xyzw
for cur, child, files in os.walk(path):
    # 'hand.npy', 'gaze.npy', 'timestamps.npy', 'contact.npy', 'head.npy', 'annotations_P0001_4bf4e21a.json', 'handmeshs.npy', 'masks.npy', 'handjoints.npy', 'objects.npy'
    timestamp = np.load(osp.join(cur, 'timestamps.npy'))
    gaze = np.load(osp.join(cur, 'gaze.npy'))    # gaze_direction (3) + gaze_center_in_world (3)
    hand = np.load(osp.join(cur, 'hand.npy'))    # left_hand (22) + right_hand (22), hand = wrist_pose (7, translation (3) + rotation (4)) + joint_angles (15)
    handjoints = np.load(osp.join(cur, 'handjoints.npy'))    # left_hand (20*3) + right_hand (20*3)
    handmeshs = np.load(osp.join(cur, 'handmeshs.npy'))
    head =np.load(osp.join(cur, 'head.npy'))     # head_direction (3) + head_translation (3) + head_rotation (4, quat_xyzw)
    objects = np.load(osp.join(cur, 'objects.npy'))  # object_data (8) * 6 objects (at most 6 objects), object_data = object_uid (1) + object_pose (7, translation (3) + rotation (4))         
    masks = np.load(osp.join(cur, 'masks.npy'))  # 3600, 4: obj, hand, head, qa
    contact = np.load(osp.join(cur, 'contact.npy')) # 3600, 4: [obj_uid, distance], left, right
    text = np.load(osp.join(cur, 'annotations.npy')) # 3600, 4: [obj_uid, distance], left, right


('../hot3dgaze', ['P0001_4bf4e21a', 'P0003_c701bd11'], [])
('../hot3dgaze/P0001_4bf4e21a', [], ['hand.npy', 'gaze.npy', 'timestamps.npy', 'contact.npy', 'head.npy', 'annotations_P0001_4bf4e21a.json', 'handmeshs.npy', 'masks.npy', 'handjoints.npy', 'objects.npy'])
('../hot3dgaze/P0003_c701bd11', [], ['hand.npy', 'gaze.npy', 'timestamps.npy', 'contact.npy', 'head.npy', 'handmeshs.npy', 'annotations_P0003_c701bd11.json', 'masks.npy', 'handjoints.npy', 'objects.npy'])


In [1]:
%load_ext autoreload 
%autoreload 2
from draft import processing_data

lcf_idx, lcov_idx, lchj_idx, ldist_value = processing_data()

  0%|          | 0/1 [00:00<?, ?it/s]

[torch.Size([3600, 1024, 16]), torch.Size([67536]), torch.Size([67536]), torch.Size([67536]), torch.Size([67536])]


100%|██████████| 1/1 [00:00<00:00,  1.04it/s]


In [7]:
[x.shape for x in [lcf_idx, lcov_idx, lchj_idx, ldist_value]]

[torch.Size([67536]),
 torch.Size([67536]),
 torch.Size([67536]),
 torch.Size([67536])]

In [4]:
import glob
import json

texts = {}
for fp in glob.glob('/home/acc/hot3d/Text2HOI/data/grab/*.json'):
    with open(fp, 'r') as f:
        fname = fp.split('/')[-1].split('.')[0]
        texts[fname] = json.load(f)

texts.keys()

dict_keys(['text_count', 'text', 'text_length'])

In [2]:
import numpy as np
data = np.load('/home/acc/hot3d/Text2HOI/data/hot3d/data.npz', allow_pickle=True)
print(data.files)

['active_flag', 'obj_exist', 'luid_nearest', 'ruid_nearest', 'x_lhand', 'x_rhand', 'x_obj', 'j_lhand', 'j_rhand', 'lhand_beta', 'rhand_beta', 'lhand_org', 'rhand_org', 'mask_hand', 'mask_object', 'mask_global', 'nframes', 'all_objs', 'is_lhand', 'is_rhand', 'lcf_idx', 'lcon_idx', 'lcov_idx', 'lchj_idx', 'ldist_value', 'rcf_idx', 'rcon_idx', 'rcov_idx', 'rchj_idx', 'rdist_value', 'object_name', 'action_name', 'hoi_name']


In [3]:
all_act = set()
for act in data['action_name']:
    all_act.update(act)

In [4]:
print(sorted(all_act))

['clean the whiteboard', 'cook', 'do exercise', 'drink coffee', 'hold', 'make a call', 'move', 'pick up', 'play', 'pour coffee', 'pour the ingredient', 'put down', 'reach for', 'rotate', 'take photo', 'type', 'use', 'write on the whiteboard']


In [5]:
print(data['hoi_name'].shape, data['nframes'].shape)
print(data['hoi_name'][0])
print(data['nframes'][0])
print(np.max(data['nframes']))

(125,) (125,)
left+pick up+cellphone|both+make a call+cellphone|right+put down+cellphone
244
469


In [ ]:
# print(data['ldist_value'][22].shape)
# print(data['ldist_value'][22])
print(data['lcf_idx'][22].shape)
print(data['lcf_idx'][22])
print(np.unique(data['lcf_idx'][22]))

(13348,)
[ 0  0  0 ... 86 86 86]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86]


In [1]:
import numpy as np
data = np.load('/home/acc/hot3d/Text2HOI/data/grab/data.npz', allow_pickle=True)
print(data.files)

['x_lhand', 'x_rhand', 'j_lhand', 'j_rhand', 'x_obj', 'lhand_beta', 'rhand_beta', 'lhand_org', 'rhand_org', 'lcf_idx', 'lcov_idx', 'lchj_idx', 'ldist_value', 'rcf_idx', 'rcov_idx', 'rchj_idx', 'rdist_value', 'is_lhand', 'is_rhand', 'object_idx', 'action', 'action_name', 'nframes', 'subject', 'background']


In [6]:
with np.load(data_path, allow_pickle=True) as data:
    # self.all_obj_uids = data["all_objs"]
    self.luid_nearest = data["luid_nearest"]
    self.ruid_nearest = data["ruid_nearest"]
    self.x_lhand = data["x_lhand"]
    self.x_rhand = data["x_rhand"]
    self.x_obj = data["x_obj"]
    self.lhand_org = data["lhand_org"]
    self.rhand_org = data["rhand_org"]
    # self.lcf_idx = data["lcf_idx"] # left hand contact frame idx
    # self.lcon_idx = data["lcon_idx"] # left contact object number idx
    # self.lcov_idx = data["lcov_idx"] # left contact object verts idx
    # self.lchj_idx = data["lchj_idx"] # left contact hand joints idx
    # self.ldist_value = data["ldist_value"]
    # self.rcf_idx = data["rcf_idx"] # right hand contact frame idx
    # self.rcon_idx = data["lcon_idx"] # right contact object number idx
    # self.rcov_idx = data["rcov_idx"] # right contact object verts idx
    # self.rchj_idx = data["rchj_idx"] # right contact hand joints idx
    # self.rdist_value = data["rdist_value"]
    # self.is_lhand = data["is_lhand"]
    # self.is_rhand = data["is_rhand"]
    # self.hoi_name = data["hoi_name"]
    # self.nframes = data["nframes"]

{'0': [0, 1], '1': [1, 2], '2': [2, 3], '3': [3, 4], '4': [4, 5]}
[0, 1]
0
1


In [1]:
from constants.hot3d_constants import hot3d_obj_name
from lib.datasets.hot3d import SequenceHOT3D, ContactHOT3D, MotionHOT3D


# Dataset = SequenceHOT3D
# Dataset = ContactHOT3D
Dataset = MotionHOT3D
ds = Dataset(
    data_path="data/hot3d/data.npz", 
    data_obj_pc_path="data/hot3d/obj.pkl", 
    text_json="data/hot3d/text.json", 
    obj_name=hot3d_obj_name, 
    max_frames=150, 
    data_ratio=1.0, 
    augm=False, 
)

data = ds[0]

Start to read data hot3d!!!
Finish to read data hot3d!! 0.04s
length of data: 25


In [2]:
data['valid_mask_obj'].shape

(150, 6)

In [1]:
from lib.networks.texthom import PositionalEncoding, HandObjectEncoding

fpe = PositionalEncoding(512)
hoipe = HandObjectEncoding(512)
print(fpe.pe.shape, hoipe.pe.shape)
print(hoipe.lhand_idx, hoipe.rhand_idx, hoipe.obj_idx)

torch.Size([5000, 1, 512]) torch.Size([2400, 1, 512])
0 300 [600, 900, 1200, 1500, 1800, 2100]


In [2]:
import torch

x = torch.rand([150, 8, 2, 512])
emb = torch.zeros([1, 1, 2, 512])

xx, emb = fpe(x, emb)
xxx = hoipe(xx)

In [24]:
a = torch.arange(36).reshape(2, 3, 6)
b = torch.zeros_like(a)
c = torch.nn.functional.pad(torch.eye(2), [0,1], 'constant', 0).unsqueeze(-1).bool()#.expand_as(a)

In [26]:
torch.where(c, b, a)

tensor([[[ 0,  0,  0,  0,  0,  0],
         [ 6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17]],

        [[18, 19, 20, 21, 22, 23],
         [ 0,  0,  0,  0,  0,  0],
         [30, 31, 32, 33, 34, 35]]])

In [25]:
print(a)
print(c)

tensor([[[ 0,  1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17]],

        [[18, 19, 20, 21, 22, 23],
         [24, 25, 26, 27, 28, 29],
         [30, 31, 32, 33, 34, 35]]])
tensor([[[ True],
         [False],
         [False]],

        [[False],
         [ True],
         [False]]])
